In [1]:
%matplotlib inline

# Introducción a Pytorch

--------------

## Carga de librerías

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

--------------

## Datasets y Dataloaders

Un Dataset es un conjunto de datos etiquetados, por ejemplo un montón de fotos etiquetadas. Pytorch ofrece varios Datasets (como FashionMNIST)

Un Dataloader es un iterable que envuelve los Datasets en batches (grupos) para poder cargar uno de estos batches, metérselo a la red neuronal y así poder trabajar con varios datos del Dataset en paralelo

Cuando usamos un Dataset de Pytorch tenemos varias opciones:
 * root: elegimos la carpeta donde se va a guardar
 * train: indicamos si queremos usar la parte del dataset de entrenamiento o de test
 * download: descargar el dataset si no está descargado
 * transform: Aplicar transformaciones a los datos
 * target_transform: Aplicar transformaciones a las etiquetas
 

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

26422272it [01:08, 383360.28it/s]                              


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



29696it [00:00, 316927.40it/s]           


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



4422656it [00:09, 471305.48it/s]                             


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



6144it [00:00, 1047169.89it/s]          
/home/mfnunez/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554788289/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


Una vez tenemos el Dataset lo convertimos en un Dataloader, indicando el batch size de cada batch

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


--------------




## Creando el modelo

En primer lugar vemos si tenemos disponible una GPU y si es así definimos esta como el dispositivo donde vamos a hacer el procesamiento

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


Creamos la red neuronal

Para esto vamos a utilizar el módulo _nn.Module_
Cada vez que creamos una red neuronal en Pytorch es necesario crear dos métodos como mínimo
 * El método __init()__ en el que se inicializa _nn.Module_ y se declara cómo va a ser la arquitectura de la red neuronal
 * El método __forward()__ que describe cómo van a pasar los datos por la red neuronal

In [6]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()   # Se inicializa el módulo nn.Module
        self.flatten = nn.Flatten()             # Se crea una primera capa que aplana la imagen de entrada
        self.linear_relu_stack = nn.Sequential( # Se crea una arquitectura secuencial:
            nn.Linear(28*28, 512),                  # Se añade una primera capa lineal que está preparada para que le entre un vector de 28*28 (784)
                                                    # y sacará un vector de 512
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 512),                    # Se añade una segunda capa lineal que le entran 512 datos y saca 512 datos
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 10)                      # Se añade una tercera capa lineal que le entran 512 datos y saca un array de tamaño 10 (el número
                                                    # de etiquetas)
        )

    def forward(self, x):
        x = self.flatten(x)                         # Se pasa la imagen por la capa de aplanado para aplanar la imagen
        logits = self.linear_relu_stack(x)          # Se pasa el vector resultante por la red
        return logits

model = NeuralNetwork().to(device)                  # Se instancia el modelo
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


--------------




## Optimización de los parámetros

Todo entrenamiento de una red neuronal consiste en la optimización de los parametros de esta para minimizar (o maximizar) una función de coste.
Se define la función de coste y la manera de optimizar (el optimizador) los parametros

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

--------------

## Definición de los bucles de entrenamiento y test

En cada época de entrenamiento se evaluan todos los batches del Dataloader

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)                  # Número de datos del dataset
    model.train()                                   # Se pone el modelo en modo entrenamiento

    for batch, (X, y) in enumerate(dataloader):     # For para obtener los datos y etiquetas de cada batch
        
        X, y = X.to(device), y.to(device)           # Se mandan los datos y las etiquetas al dispositivo (en este caso la GPU)

        # Compute prediction error
        pred = model(X)                             # Se obtienen las salidas de la red para los datos
        loss = loss_fn(pred, y)                     # Se calcula la pérdida de estas salidas con respecto a las etiquetas en función
                                                    # de la función de pérdida elegida

        # Backpropagation
        optimizer.zero_grad()                       # Se ponen a cero los gradientes
        loss.backward()                             # Se obtienen los gradientes de los parámetros
        optimizer.step()                            # Se actualizan los parámetros

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)                 # Se calcula la pérdida y el número de datos procesados
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")      # Se imprime la pérdida y el número de datos procesados de todos los que hay en un batch

Durante el test se evaluan los datos de test



In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)              # Número de datos del dataset
    num_batches = len(dataloader)               # Número de batches
    model.eval()                                # Se pone el modelo en modo evaluación
    test_loss, correct = 0, 0                   # Se inicializan las variables de pérdida y accuracy

    with torch.no_grad():                                                       # Se indica que no se calculen los gradientes
        for X, y in dataloader:                                                 # Se obtienen las imágenes y etiquetas del dataloader
            X, y = X.to(device), y.to(device)                                   # Se mandan al dispositivo (en este caso la GPU)

            pred = model(X)                                                     # Se obtienen las predicciones del modelo

            test_loss += loss_fn(pred, y).item()                                # Se acumula el error
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()     # Se acumula cada vez que se acierta uno

    test_loss /= num_batches                                                                    # Se calcula el error dividiendo el total entre el número de batches
    correct /= size                                                                             # Se obtiene el accuracy dividiendo los resultados buenos entre el 
                                                                                                # total de datos
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")     # Se imprime el accuracy y el error medio

--------------

## Entrenamiento

Se entrena durante varias épocas

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308877  [    0/60000]
loss: 2.291079  [ 6400/60000]
loss: 2.265797  [12800/60000]
loss: 2.255189  [19200/60000]
loss: 2.246202  [25600/60000]
loss: 2.215523  [32000/60000]
loss: 2.219025  [38400/60000]
loss: 2.189023  [44800/60000]
loss: 2.182710  [51200/60000]
loss: 2.148128  [57600/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 2.142635 

Epoch 2
-------------------------------
loss: 2.156960  [    0/60000]
loss: 2.142013  [ 6400/60000]
loss: 2.076811  [12800/60000]
loss: 2.095929  [19200/60000]
loss: 2.047673  [25600/60000]
loss: 1.987178  [32000/60000]
loss: 2.014789  [38400/60000]
loss: 1.934459  [44800/60000]
loss: 1.931124  [51200/60000]
loss: 1.861800  [57600/60000]
Test Error: 
 Accuracy: 62.1%, Avg loss: 1.856072 

Epoch 3
-------------------------------
loss: 1.892417  [    0/60000]
loss: 1.855688  [ 6400/60000]
loss: 1.730256  [12800/60000]
loss: 1.779484  [19200/60000]
loss: 1.668417  [25600/60000]
loss: 1.625547  [32000/600

--------------




## Guardar el modelo

Se pueden guardar los parámetros del modelo

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


--------------

## Carga del modelo

Se pueden cargar los parámetros de un modelo previamente guardados, para ello primero hay que hacer una instancia del modelo

In [13]:
model = NeuralNetwork()                             # Se instancia el modelo
model.load_state_dict(torch.load("model.pth"))      # Se cargan sus parámetros

<All keys matched successfully>

--------------

## Predicciones

Se puede usar el modelo para hacer predicciones



In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
